Processing the data
使用範例資料集
安裝套件

In [1]:
!pip install datasets evaluate transformers[sentencepiece]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 10.6 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


載入Huggingface中的資料集
Huggingface中的資料集，透過Datasets類別下載
這邊使用GLUE資料集中的MRPC子資料集作為範例
GLUE 基準是一個學術基準，用於衡量機器學習模型
MRPC資料集是構成GLUE的10個資料集之一，由5801對句子組成，並帶有一個標籤，表示兩句話是否相似

In [2]:
from datasets import load_dataset

raw_datasets = load_dataset("glue", "mrpc")
raw_datasets


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

得到一個DatasetDict包含訓練集、驗證集和測試集的物件。其中每一個都包含幾列 ( sentence1、sentence2、label和idx) 和多行，這些行是每個集合中的元素數量（訓練集中有3668對句子，驗證集中有408對句子，以及1725對句子在測試集中）。

此命令下載並快取資料集，預設位於~/.cache/huggingface/datasets中

透過索引來存取物件中的每對句子raw_datasets

In [3]:
raw_train_dataset = raw_datasets["train"]
raw_train_dataset[0]

{'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .',
 'label': 1,
 'idx': 0}

標籤已經是整數，因此不必進行任何預處理。要知道哪個整數對應於哪個標籤，可以用features檢查raw_train_dataset，會顯示每列的類型。label類型為ClassLabel，整數到標籤名稱的對應儲存在名稱資料夾中。0對應於not_equivalent, 且1對應於equivalent.

In [4]:
raw_train_dataset.features

{'sentence1': Value(dtype='string', id=None),
 'sentence2': Value(dtype='string', id=None),
 'label': ClassLabel(names=['not_equivalent', 'equivalent'], id=None),
 'idx': Value(dtype='int32', id=None)}

為了預處理資料集，我們需要將文字轉換為模型可以理解的數字。這是透過tokenizer完成的。可以向tokenizer提供一個句子或句子列表，因此我們可以直接對每對的所有第一個句子和所有第二個句子進行分詞，如下所示：

In [5]:
from transformers import AutoTokenizer

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tokenized_sentences_1 = tokenizer(raw_datasets["train"]["sentence1"])
tokenized_sentences_2 = tokenizer(raw_datasets["train"]["sentence2"])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [8]:
inputs = tokenizer("This is the first sentence.", "This is the second one.")
inputs

{'input_ids': [101, 2023, 2003, 1996, 2034, 6251, 1012, 102, 2023, 2003, 1996, 2117, 2028, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [9]:
tokenizer.convert_ids_to_tokens(inputs["input_ids"])

['[CLS]',
 'this',
 'is',
 'the',
 'first',
 'sentence',
 '.',
 '[SEP]',
 'this',
 'is',
 'the',
 'second',
 'one',
 '.',
 '[SEP]']

對應輸入部分的[CLS], sentence1 ID 全部為0，而對應剩餘部分的sentence2[SEP]類型ID全部為1

注意，不同的checkpoint，不一定包含token_type_ids（如果DistilBERT模型則沒有）。只有當模型知道如何處理它們時才會有，因為預訓練期間已經就已經加入了。不需要擔心token_type_ids標記化輸入中是否存在：只要對標記化器和模型使用相同的檢查點即可

接下來將兩個句子的資料傳給tokenizer進行處理

In [10]:
tokenized_dataset = tokenizer(
    raw_datasets["train"]["sentence1"],
    raw_datasets["train"]["sentence2"],
    padding=True,
    truncation=True,
)

使用Dataset.map()方法。如果需要進行更多的預處理而不僅僅是標記化，這也提供了一些額外的靈活性。此map()方法的工作原理是對資料集的每個元素應用一個函數，定義一個對輸入進行標記的函數

In [11]:
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

函數接受一個字典（就像我們資料集的項目）並傳回一個帶有鍵input_ids、attention_mask和 的新字典token_type_ids。請注意，如果example字典包含多個樣本（每個鍵作為句子列表），它也適用，因為它tokenizer適用於句子對列表，如前所述。這將允許我們batched=True在呼叫中使用該選項map()，這將大大加快標記化速度。它由Tokenizers tokenizer庫中用 Rust 編寫的 tokenizer 支援。這個分詞器可以非常快，但前提是一次大量輸入

In [12]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
tokenized_datasets

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})

In [ ]:
在套用預處理函數時，您甚至可以map()透過傳遞num_proc參數來使用多重處理。我們在這裡沒有這樣做，因為Tokenizers 庫已經使用多個線程來更快地標記我們的樣本，但如果您沒有使用此庫支援的快速標記生成器，這可以加快您的預處理速度。如果的預處理函數為我們應用的資料集中的現有鍵傳回新值，我們也可以變更現有欄位。

一般將資料送入模型時是一批一批的，如果一次將資料庫的所有資料都做padding或truncate會很浪費時間，每一批資料送入模型再進行處理較好。

注意，我們暫時將參數保留在標記化函數中。這是因為將所有樣本填充到最大長度效率不高：最好在建立批次時填充樣本，因為這樣我們只需要填充到該批次中的最大長度，而不是整個資料集。當輸入的長度變化很大時，這可以節省大量時間和處理能力！

動態填充
負責將樣本放在批次中的函數稱為collection function。您在建立時可以傳遞的參數DataLoader，預設值是一個函數，將樣本轉換為 PyTorch 張量並連接它們. 因為輸入不會全部有相同的大小。我們故意延遲填充，以便僅在每個批次上根據需要應用它，並避免出現過長的輸入和大量填充。這將大大加快訓練速度

為了在實踐中做到這一點，必須定義一個collection function，正確數量的填充應用於批次在一起的資料集的項目。Transformers庫通過DataCollatorWithPadding.實例化，它需要一個標記生成器（以了解要使用哪個及模型是否期望填充位於輸入的左側或右側），並將執行所有操作

In [13]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

取得訓練集中的前8筆資料，並且將不要的欄位去除(將文字去除，只留下數字欄位，也將idx去掉因為沒用)

算出批次8中每一個序列的長度，並且將這些長度放在另一個list中

In [14]:
samples = tokenized_datasets["train"][:8]
samples = {k: v for k, v in samples.items() if k not in ["idx", "sentence1", "sentence2"]}
[len(x) for x in samples["input_ids"]]

[50, 59, 47, 67, 59, 50, 62, 32]

In [15]:
batch = data_collator(samples)
{k: v.shape for k, v in batch.items()}

{'input_ids': torch.Size([8, 67]),
 'token_type_ids': torch.Size([8, 67]),
 'attention_mask': torch.Size([8, 67]),
 'labels': torch.Size([8])}